In [1]:
import ee
import geemap
import collections
collections.Callable = collections.abc.Callable
ee.Initialize()

In [52]:
Map = geemap.Map()

bangalore = ee.FeatureCollection("users/ujavalgandhi/public/bangalore_boundary")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
# The following collections were created using the
# Drawing Tools in the code editor
urban = ee.FeatureCollection("users/ujavalgandhi/e2e/urban_gcps")
bare = ee.FeatureCollection("users/ujavalgandhi/e2e/bare_gcps")
water = ee.FeatureCollection("users/ujavalgandhi/e2e/water_gcps")
vegetation = ee.FeatureCollection("users/ujavalgandhi/e2e/vegetation_gcps")

filtered = s2 \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.bounds(bangalore)) \
  .select('B.*')

composite = filtered.median().clip(bangalore)

# Display the input composite.
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}
Map.addLayer(composite, rgbVis, 'image')

gcps = urban.merge(bare).merge(water).merge(vegetation)

# Overlay the point on the image to get training data.
training = composite.sampleRegions(
  collection=gcps, properties=['landcover'], scale=10)

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(
  features=training, classProperty='landcover', inputProperties=composite.bandNames())

# # Classify the image.
classified = composite.classify(classifier)
Map.addLayer(classified, {'min': 0, 'max': 3, 'palette': ['gray', 'brown', 'blue', 'green']}, '2019')

# Display the GCPs
# We use the style() function to style the GCPs
palette = ee.List(['gray','brown','blue','green'])
landcover = ee.List([0, 1, 2, 3])

def funclc(lc):
    color = palette.get(landcover.indexOf(lc))
    markerStyle = {'color': 'white', 'pointShape': 'diamond', \
        'pointSize': 4, 'width': 1, 'fillColor': color}
    def funcp(point):
        return point.set('style', markerStyle)
    return gcps.filter(ee.Filter.eq('landcover', lc)) \
        .map(funcp)
    
gcpsStyled = ee.FeatureCollection(
    landcover.map(funclc)).flatten()

gcpsStyling = gcpsStyled.style(
    styleProperty='style')

Map.addLayer(gcpsStyling, {}, 'GCPs')
Map.centerObject(gcpsStyled)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
Map = geemap.Map()

s2 = ee.ImageCollection("COPERNICUS/S2_SR")
urbanAreas = ee.FeatureCollection("users/ujavalgandhi/e2e/ne_10m_urban_areas")

# Perform supervised classification for your city
# Find the feature id by adding the layer to the map and using Inspector.
city = urbanAreas.filter(ee.Filter.eq('system:index', '00000000000000002bf8'))
geometry = city.geometry()
Map.centerObject(geometry)

filtered = s2 \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.bounds(geometry)) \
  .select('B.*')

composite = filtered.median().clip(geometry)

# Display the input composite.

rgbVis = {'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
Map.addLayer(composite, rgbVis, 'image')

# Exercise
# Add training points for 4 classes
# Assign the 'landcover' property as follows

# urban: 0
# bare: 1
# water: 2
# vegetation: 3

# After adding points, uncomments lines below

# gcps = urban.merge(bare).merge(water).merge(vegetation)

# # Overlay the point on the image to get training data.
# training = composite.sampleRegions({
#   collection: gcps,
#   properties: ['landcover'],
#   scale: 10,
#   tileScale: 16
# })
# print(training)

# # Train a classifier.
# classifier = ee.Classifier.smileRandomForest(50).train({
#   features: training,
#   classProperty: 'landcover',
#   inputProperties: composite.bandNames()
# })
# # # Classify the image.
# classified = composite.classify(classifier)
# Map.addLayer(classified, {min: 0, max: 3, palette: ['gray', 'brown', 'blue', 'green']}, '2019')
Map